In [1]:
! pip install kaleido

In [2]:
!pip install tensorflow

In [1]:
import numpy as np
import keras
import os
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tarfile
import zipfile
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import AgglomerativeClustering
from scipy.io import loadmat
from urllib.request import urlretrieve
from os.path import isfile, isdir
import plotly.figure_factory as ff
import plotly.express as px
import statsmodels.api as sm
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
import glob
import random
import matplotlib.image as img
from PIL import Image
#Install pillow
from PIL import Image
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [175]:
desire_size = (50, 50)
n_channels = 3
batch_size = 128

Se define la ruta de la carpeta que contiene las imágenes de entrenamiento y la lista de nombres de las carpetas de categorías.

In [176]:
train_folder = 'data/train'
categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen',
              'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse',
              'Small-flowered Cranesbill', 'Sugar beet']

Se inicializan listas para almacenar las imágenes y las etiquetas

In [177]:
images = []
labels = []

Se itera sobre las carpetas de categorías

In [178]:
for category in categories:
    folder_path = os.path.join(train_folder, category)
    # Se itera sobre los archivos de imagen en cada carpeta
    for file_name in os.listdir(folder_path):
        # Se lee la imagen y convertirla a escala de grises
        image_path = os.path.join(folder_path, file_name)
        image = Image.open(image_path).convert('L')
        # Se redimensiona la imagen a un tamaño fijo, por ejemplo (128, 128)
        image = image.resize((128, 128))
        # Se convierte la imagen a un array numpy
        image_array = np.array(image)
        # Se agrega la imagen y la etiqueta a las listas
        images.append(image_array)
        labels.append(category)

Se convierten las listas a arrays numpy

In [179]:
images = np.array(images)
labels = np.array(labels)

Se codifican las etiquetas numéricamente

In [180]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [181]:
num_out = len(label_encoder.classes_)

Se convierten las etiquetas a variables categóricas

In [182]:
labels_categorical = to_categorical(labels_encoded)

Se dividen los datos en conjuntos de entrenamiento y prueba

In [183]:
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

Se normalizan los valores de píxel entre 0 y 1

In [184]:
X_train = X_train / 255.0
X_test = X_test / 255.0

Se define la arquitectura del modelo de red convolucional

In [185]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1), padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_out, activation='softmax'))

Se compila el modelo

In [186]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Se entrena el modelo con 10 epochs

In [214]:
model.fit(X_train, y_train, epochs=4, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/4
238/238 [==============================] - 82s 346ms/step - loss: 0.3416 - accuracy: 0.8853 - val_loss: 3.4988 - val_accuracy: 0.3263
Epoch 2/4
238/238 [==============================] - 80s 335ms/step - loss: 0.2438 - accuracy: 0.9226 - val_loss: 3.6635 - val_accuracy: 0.3453
Epoch 3/4
238/238 [==============================] - 74s 313ms/step - loss: 0.1982 - accuracy: 0.9403 - val_loss: 3.9844 - val_accuracy: 0.2874
Epoch 4/4
238/238 [==============================] - 78s 329ms/step - loss: 0.1708 - accuracy: 0.9474 - val_loss: 3.8434 - val_accuracy: 0.3084


Se Clasifican las imágenes de prueba y generar el archivo CSV

In [215]:
test_folder = 'data/test'
test_images = []

Se leen las imágenes de prueba y preprocesarlas

In [216]:
for file_name in os.listdir(test_folder):
    image_path = os.path.join(test_folder, file_name)
    image = Image.open(image_path).convert('L')
    image = image.resize((128, 128))
    image_array = np.array(image)
    test_images.append(image_array)

Se convierten las imágenes de prueba a un array numpy y normalizar los valores de píxel

In [217]:
test_images = np.array(test_images) / 255.0

Se realizan las predicciones en las imágenes de prueba

In [218]:
predictions = model.predict(test_images)

25/25 [==============================] - 2s 93ms/step


Se obtienen las etiquetas predichas para cada imagen

In [219]:
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

Se genera un DataFrame con los nombres de archivo y las etiquetas predichas

In [220]:
results_df = pd.DataFrame({'file': os.listdir(test_folder), 'species': predicted_labels})

Se guardan los resultados en un archivo CSV

In [221]:
predictions_file = os.path.join('data', 'predictions.csv')
results_df.to_csv(predictions_file, index=False)

Se muestran los resultados

In [222]:
predictions_data = pd.read_csv(predictions_file)
print(predictions_data)

              file                    species
0    0021e90e4.png                Black-grass
1    003d61042.png           Loose Silky-bent
2    007b3da8b.png                      Maize
3    0086a6340.png           Common Chickweed
4    00c47e980.png           Loose Silky-bent
..             ...                        ...
789  fea355851.png           Common Chickweed
790  fea3da57c.png           Loose Silky-bent
791  fef2ade8c.png           Loose Silky-bent
792  ff65bc002.png  Small-flowered Cranesbill
793  ffc6f8527.png                      Maize

[794 rows x 2 columns]
